In [3]:
import codecs
import os
tag_dic = {"has_clinical_manifestation":"临床表现",
          "cause":"病因",
          "diagnose":"诊断",
          "medical_treat":"药物治疗",
          "prevent":"预防",
          "is_synonym":"同义",
          "is_pathology":"病理",
          "nurse":"护理",
          "report":"报告",
          "inspect":"观察",
          "non-medication_treat":"非药治疗",
          "is_complication":"并发症",
          "surgical_treat":"手术治疗",
          "disease": "疾病",
           "drug": "药物",
           "surgery": "手术",
           "symptom": "症状",
           "prevention": "预防措施",
           "inspection_method": "观察方法",
           "diagnostic_technology": "诊断技术",
           "non-drug_therapy": "非药疗法",
           "nursing_technology": "护理技术",
           "cause_of_disease": "病因",
           "pathological_mechanism": "病理",
           'medical_indicator': '医疗指标'
          }
 # 由于brat不支持中文标签 所以tag_dic 将标签转化成中文的
def from_ann2dic(r_ann_path, r_txt_path, w_path):
    q_dic = {}
    print("开始读取文件:%s" % r_ann_path)
    with codecs.open(r_ann_path, "r", encoding="utf-8") as f:#读取标签文件
        line = f.readline()
        line = line.strip("\n\r")
        while line != "":
            line_arr = line.split()
            print(line_arr)
            cls = tag_dic[line_arr[1]] #line_arr[1]为标签名称 通过tag_dic 查字典后转化为中文标签
            try:
                start_index = int(line_arr[2])#标签起始位置
                end_index = int(line_arr[3])#标签的结束位置
                length = end_index - start_index#标签长度
                for r in range(length):#为标签内的字标号
                    if r == 0:#第一个字标B
                        q_dic[start_index] = ("B-%s" % cls) #在字典内添加字标签 eg: 21：B-药物
                    else:#标签内其余的标I
                        q_dic[start_index + r] = ("I-%s" % cls)#在字典内添加字标签 eg: 22：I-药物
            except:
                line = f.readline()
                line = line.strip("\n\r")
                continue#出现异常则跳过 继续循环
            line = f.readline()
            line = line.strip("\n\r")
    print("开始读取文件:%s" % r_txt_path)
    with codecs.open(r_txt_path, "r", encoding="utf-8") as f:#读取文本文档
        content_str = f.read()
        # content_str = content_str.replace("\n", "").replace("\r", "").replace("//////", "\n")
    print("开始写入文本%s" % w_path)#开始将标签写入文本
    with codecs.open(w_path, "w", encoding="utf-8") as w:
        for i, str in enumerate(content_str):
            if str is " " or str == "" or str == "\n" or str == "\r":
                print("===============")
            elif str == "/":
                if i == len(content_str) - len("//////") + 1:  # 表示到达末尾
                    # w.write("\n")
                    break
                # 连续六个字符首尾都是/,则表示换一行
                elif content_str[i + len("//////") - 1] == "/" and content_str[i + len("//////") - 2] == "/" and \
                        content_str[i + len("//////") - 3] == "/" and content_str[i + len("//////") - 4] == "/" and \
                        content_str[i + len("//////") - 5] == "/":
                        w.write("\n")#`123
                        i += len("//////")
            else:
                if i in q_dic:#文本中的位置若存在标签 赋给tag标签
                    tag = q_dic[i]
                else:
                    tag = "O"  # 大写字母O
                w.write('%s\t%s\n' % (str, tag))#每个字一行每个字都有标签 分隔符在此设置
                if str=="。"or str=="?"or str=="!":#输入要求 保证每一句话后都有一个空格
                    w.write("\n")
        w.write('%s\n' % "END O")
#去除空行
def drop_null_row(r_path, w_path):
    q_list = []
    with codecs.open(r_path, "r", encoding="utf-8") as f:
        line = f.readline()
        line = line.strip("\n\r")
        while line != "END O":
            if line != "":
                q_list.append(line)
            line = f.readline()
            line = line.strip("\n\r")
    with codecs.open(w_path, "w", encoding="utf-8") as w:
        for i, line in enumerate(q_list):
            w.write('%s\n' % line)
# 生成train.txt、dev.txt、test.txt
# 除10-11-new.txt分别用于dev和test外,剩下的合并成train.txt
def rw0(data_root_dir, w_path):
    if os.path.exists(w_path):
        os.remove(w_path)
    for file in os.listdir(data_root_dir):
        path = os.path.join(data_root_dir, file)
        if file.endswith("abstract_010-new.txt"):#生成的文件的名字
            # 重命名为dev.txt
            os.rename(path, os.path.join(data_root_dir, "dev.txt"))
            continue
        if file.endswith("abstract_011-new.txt"):
            # 重命名为test.txt
            os.rename(path, os.path.join(data_root_dir, "test.txt"))
            continue
        q_list = []
        print("开始读取文件:%s" % file)
        with codecs.open(path, "r", encoding="utf-8") as f:
            line = f.readline()
            line = line.strip("\n\r")
            while line != "END O":
                q_list.append(line)
                line = f.readline()
                line = line.strip("\n\r")
        print("开始写入文本%s" % w_path)
        with codecs.open(w_path, "a", encoding="utf-8") as f:
            for item in q_list:
                if item.__contains__('\ufeff1'):
                    print("===============")
                f.write('%s\n' % item)
if __name__ == '__main__':
    data_dir = "datas"
    for file in os.listdir(data_dir):
        if file.find(".") == -1:
            continue
        file_name = file[0:file.find(".")]
        r_ann_path = os.path.join(data_dir, "%s.ann" % file_name)
        r_txt_path = os.path.join(data_dir, "%s.txt" % file_name)
        w_path = "%s/new/%s-new.txt" % (data_dir, file_name)
        from_ann2dic(r_ann_path, r_txt_path, w_path)
    # 生成train.txt、dev.txt、test.txt
    rw0("%s/new" % data_dir, "%s/new/train.txt" % data_dir)

开始读取文件:datas\abstract_000.ann
['T1', 'disease', '250', '257', '阿尔茨海默氏病']
['T2', 'symptom', '283', '288', '进行性痴呆']
['T3', 'cause_of_disease', '354', '356', '遗传']
['R1', 'has_clinical_manifestation', 'Arg1:T1', 'Arg2:T2']
['R2', 'cause', 'Arg1:T3', 'Arg2:T1']
['T4', 'disease', '24', '30', '阿尔茨海默病']
['T5', 'diagnostic_technology', '38', '41', 'PET']
['R3', 'diagnose', 'Arg1:T5', 'Arg2:T4']
['T6', 'drug', '713', '718', '补肾健脾方']
['T7', 'disease', '561', '567', '阿尔茨海默病']
['T8', 'drug', '1520', '1523', '奎硫平']
['T9', 'drug', '1524', '1527', '利培酮']
['T10', 'disease', '1528', '1534', '阿尔茨海默病']
['R5', 'medical_treat', 'Arg1:T8', 'Arg2:T10']
['R6', 'medical_treat', 'Arg1:T9', 'Arg2:T10']
['T11', 'disease', '2565', '2571', '阿尔茨海默病']
['T12', 'symptom', '2589', '2597', '大脑皮层厚度萎缩']
['T13', 'symptom', '2598', '2606', '认知功能逐步下降']
['R7', 'has_clinical_manifestation', 'Arg1:T11', 'Arg2:T12']
['R8', 'has_clinical_manifestation', 'Arg1:T11', 'Arg2:T13']
['T14', 'disease', '4734', '4740', '阿尔兹海默症']
['T15', '

开始读取文件:datas\abstract_000.ann
['T1', 'disease', '250', '257', '阿尔茨海默氏病']
['T2', 'symptom', '283', '288', '进行性痴呆']
['T3', 'cause_of_disease', '354', '356', '遗传']
['R1', 'has_clinical_manifestation', 'Arg1:T1', 'Arg2:T2']
['R2', 'cause', 'Arg1:T3', 'Arg2:T1']
['T4', 'disease', '24', '30', '阿尔茨海默病']
['T5', 'diagnostic_technology', '38', '41', 'PET']
['R3', 'diagnose', 'Arg1:T5', 'Arg2:T4']
['T6', 'drug', '713', '718', '补肾健脾方']
['T7', 'disease', '561', '567', '阿尔茨海默病']
['T8', 'drug', '1520', '1523', '奎硫平']
['T9', 'drug', '1524', '1527', '利培酮']
['T10', 'disease', '1528', '1534', '阿尔茨海默病']
['R5', 'medical_treat', 'Arg1:T8', 'Arg2:T10']
['R6', 'medical_treat', 'Arg1:T9', 'Arg2:T10']
['T11', 'disease', '2565', '2571', '阿尔茨海默病']
['T12', 'symptom', '2589', '2597', '大脑皮层厚度萎缩']
['T13', 'symptom', '2598', '2606', '认知功能逐步下降']
['R7', 'has_clinical_manifestation', 'Arg1:T11', 'Arg2:T12']
['R8', 'has_clinical_manifestation', 'Arg1:T11', 'Arg2:T13']
['T14', 'disease', '4734', '4740', '阿尔兹海默症']
['T15', '

开始读取文件:datas\abstract_001.ann
['T1', 'disease', '2', '8', '阿尔茨海默病']
['T2', 'disease', '310', '316', '阿尔茨海默病']
['T3', 'symptom', '360', '362', '偏执']
['T4', 'symptom', '363', '368', '情绪不稳定']
['T5', 'symptom', '369', '374', '无目的漫游']
['T6', 'symptom', '375', '377', '攻击']
['T8', 'symptom', '378', '380', '破坏']
['T7', 'symptom', '381', '383', '吵闹']
['R1', 'has_clinical_manifestation', 'Arg1:T2', 'Arg2:T3']
['R2', 'has_clinical_manifestation', 'Arg1:T2', 'Arg2:T4']
['R3', 'has_clinical_manifestation', 'Arg1:T2', 'Arg2:T5']
['R4', 'has_clinical_manifestation', 'Arg1:T2', 'Arg2:T6']
['R5', 'has_clinical_manifestation', 'Arg1:T2', 'Arg2:T8']
['R6', 'has_clinical_manifestation', 'Arg1:T2', 'Arg2:T7']
['T9', 'cause_of_disease', '55', '57', '基因']
['R7', 'cause', 'Arg1:T9', 'Arg2:T1']
['T11', 'disease', '255', '261', '阿尔茨海默病']
['T12', 'drug', '266', '271', '促认知药物']
['R8', 'medical_treat', 'Arg1:T12', 'Arg2:T11']
['T10', 'disease', '458', '464', '阿尔茨海默病']
['T13', 'symptom', '491', '498', '进行性认知障碍']
['

开始读取文件:datas\abstract_002.ann
['T1', 'disease', '0', '7', '阿尔茨海默氏病']
['T2', 'disease', '33', '40', '神经退行性疾病']
['R1', 'is_complication', 'Arg1:T2', 'Arg2:T1']
['T3', 'symptom', '48', '56', '细胞间淀粉样斑块']
['T4', 'symptom', '63', '69', '神经纤维缠结']
['R2', 'has_clinical_manifestation', 'Arg1:T1', 'Arg2:T3']
['R3', 'has_clinical_manifestation', 'Arg1:T1', 'Arg2:T4']
['T5', 'disease', '105', '108', '老年斑']
['T6', 'disease', '115', '117', '炎症']
['T7', 'disease', '118', '124', '神经细胞凋亡']
['T8', 'symptom', '146', '152', '学习记忆障碍']
['T9', 'symptom', '153', '157', '行为异常']
['T11', 'symptom', '224', '230', '突触功能障碍']
['T10', 'cause_of_disease', '180', '186', '可溶性寡聚体']
['T12', 'cause_of_disease', '79', '86', 'β-淀粉样蛋白']
['R4', 'cause', 'Arg1:T12', 'Arg2:T5']
['R5', 'cause', 'Arg1:T12', 'Arg2:T6']
['R6', 'cause', 'Arg1:T12', 'Arg2:T7']
['T13', 'disease', '273', '279', '阿尔茨海默病']
['T14', 'cause_of_disease', '304', '309', '胰岛素抵抗']
['R7', 'cause', 'Arg1:T14', 'Arg2:T13']
['T15', 'cause_of_disease', '310', '312', '血

开始读取文件:datas\abstract_003.ann
['T1', 'pathological_mechanism', '5', '10', '胞二磷胆碱']
['T2', 'disease', '341', '347', '阿尔茨海默病']
['T3', 'disease', '375', '381', '老年性痴呆症']
['T4', 'cause_of_disease', '384', '389', '神经元丧失']
['T5', 'cause_of_disease', '391', '398', '神经递质的缺少']
['T6', 'drug', '719', '723', '维生素B']
['T7', 'disease', '1251', '1257', '阿尔茨海默病']
['T8', 'disease', '1281', '1286', '老年性痴呆']
['T9', 'symptom', '1316', '1324', '退行性记忆力减退']
['T10', 'symptom', '1325', '1331', '认知功能障碍']
['T11', 'symptom', '1332', '1336', '人格改变']
['T12', 'prevention', '1392', '1398', '维生素B12']
['T13', 'drug', '1907', '1913', '益智健脑颗粒']
['T14', 'disease', '1914', '1920', '阿尔茨海默病']
['T15', 'inspection_method', '1974', '1987', '双抗体夹心酶联免疫吸附测定']
['T16', 'non-drug_therapy', '2286', '2291', '颈椎病治疗']
['T17', 'disease', '2292', '2297', '精神性疾病']
['T18', 'disease', '2323', '2326', '抑郁症']
['T19', 'disease', '2327', '2332', '老年痴呆症']
['T20', 'disease', '2333', '2338', '精神分裂症']
['T21', 'cause_of_disease', '2503', '2511', '甲状腺的

...

['T69', 'disease', '1614', '1619', '老年性痴呆']
['T70', 'nursing_technology', '1667', '1671', '康复护理']
['T71', 'nursing_technology', '1672', '1676', '健康教育']
['R29', 'nurse', 'Arg1:T70', 'Arg2:T69']
['R30', 'nurse', 'Arg1:T71', 'Arg2:T69']
['T72', 'nursing_technology', '1700', '1706', '日常生活护理']
['T73', 'nursing_technology', '1707', '1713', '生活自理训练']
['T74', 'nursing_technology', '1714', '1718', '精神护理']
['T75', 'nursing_technology', '1719', '1723', '安全护理']
['T76', 'symptom', '1804', '1808', '老年痴呆']
['R31', 'nurse', 'Arg1:T72', 'Arg2:T69']
['R32', 'nurse', 'Arg1:T73', 'Arg2:T69']
['R33', 'nurse', 'Arg1:T74', 'Arg2:T69']
['R34', 'nurse', 'Arg1:T75', 'Arg2:T69']
['T77', 'disease', '1811', '1817', '阿尔茨海默症']
['T78', 'disease', '1856', '1872', '慢性渐进性中枢神经系统退行性疾病']
['T79', 'disease', '1887', '1889', 'AD']
['T80', 'drug', '1901', '1908', '胆碱酯酶抑制药']
['T82', 'drug', '1909', '1912', '他克林']
['T83', 'drug', '1913', '1917', '多奈哌齐']
['T84', 'drug', '1918', '1922', '加兰他敏']
['T85', 'drug', '1923', '1928', '利伐斯

开始读取文件:datas\abstract_010.ann
['T2', 'disease', '5', '8', 'MCI']
['T3', 'disease', '9', '11', 'AD']
['T4', 'inspection_method', '16', '21', '血液流变学']
['T1', 'disease', '34', '36', 'AD']
['T5', 'disease', '40', '43', 'MCI']
['T6', 'disease', '81', '84', 'MCI']
['T7', 'disease', '86', '88', 'AD']
['T9', 'disease', '157', '160', 'MCI']
['T10', 'disease', '161', '163', 'AD']
['T11', 'disease', '185', '188', 'MCI']
['T12', 'disease', '189', '191', 'AD']
['T13', 'inspection_method', '200', '204', '血流变学']
['T14', 'disease', '209', '213', '高脂血症']
['T15', 'inspection_method', '218', '223', '血液流变学']
['T16', 'inspection_method', '224', '228', '血脂检测']
['T17', 'disease', '237', '240', 'MCI']
['R1', 'inspect', 'Arg1:T16', 'Arg2:T17']
['R2', 'inspect', 'Arg1:T15', 'Arg2:T17']
['T18', 'disease', '241', '243', 'AD']
['R3', 'inspect', 'Arg1:T16', 'Arg2:T18']
['R4', 'inspect', 'Arg1:T15', 'Arg2:T18']
['T19', 'medical_indicator', '90', '92', 'TC']
['T8', 'medical_indicator', '93', '97', 'ApoB']
['T20', 'me

开始读取文件:datas\abstract_011.ann
['T1', 'disease', '0', '7', '阿尔茨海默氏症']
['T2', 'disease', '21', '32', '中枢神经系统退行性疾病']
['T3', 'disease', '69', '75', '精神行为障碍']
['T4', 'disease', '114', '119', '老年痴呆症']
['T5', 'disease', '122', '130', '神经性退变的疾病']
['T6', 'disease', '456', '463', '阿尔茨海默氏症']
['T7', 'disease', '575', '582', '阿尔茨海默氏症']
['T8', 'disease', '779', '786', '阿尔茨海默氏症']
['T9', 'disease', '916', '921', '老年痴呆症']
['T10', 'cause_of_disease', '1001', '1008', 'β-淀粉样蛋白']
['T11', 'disease', '1079', '1085', '阿尔茨海默病']
['T12', 'nursing_technology', '1113', '1119', '强化护理干预']
['T13', 'non-drug_therapy', '1123', '1128', '脑反射治疗']
['T14', 'non-drug_therapy', '1129', '1133', '穴位按摩']
['T15', 'non-drug_therapy', '1134', '1136', '光疗']
['T16', 'non-drug_therapy', '1137', '1141', '音乐疗法']
['T17', 'inspection_method', '1180', '1188', '日常生活活动量表']
['T18', 'inspection_method', '1207', '1217', '汉语标准失语症检查法']
['T19', 'inspection_method', '1246', '1254', '生活自理能力量表']
['T20', 'inspection_method', '1261', '1272', '工具性日常生活能力

开始读取文件:abstract_000-new.txt
开始写入文本datas/new/train.txt
开始读取文件:abstract_001-new.txt
开始写入文本datas/new/train.txt
开始读取文件:abstract_002-new.txt
开始写入文本datas/new/train.txt
开始读取文件:abstract_003-new.txt
开始写入文本datas/new/train.txt
开始读取文件:abstract_004-new.txt
开始写入文本datas/new/train.txt
开始读取文件:abstract_005-new.txt
开始写入文本datas/new/train.txt
开始读取文件:abstract_006-new.txt
开始写入文本datas/new/train.txt
开始读取文件:abstract_007-new.txt
开始写入文本datas/new/train.txt
开始读取文件:abstract_008-new.txt
开始写入文本datas/new/train.txt
开始读取文件:abstract_009-new.txt
开始写入文本datas/new/train.txt


FileExistsError: [WinError 183] 当文件已存在时，无法创建该文件。: 'datas/new\\abstract_010-new.txt' -> 'datas/new\\dev.txt'

关系抽取

In [54]:
import codecs
tag_dic = {"has_clinical_manifestation":"临床表现",
           "cause":"病因",
           "diagnose":"诊断",
           "medical_treat":"治疗",
           "prevent":"预防",
           "is_synonym":"同义",
           'is_synonym_drug':"同义",
           'is_synonym_mechanism':"同义",
           'is_synonym_cause':"同义",
           'is_synonym_surgery': "同义",
           'is_synonym_symptom':"同义",
           'is_synonym_prevention':"同义",
           'is_synonym_inspection':"同义",
           'is_synonym_technology':"同义",
           'is_synonym_therapy':"同义",
           'is_synonym_nursing':"同义",
           'is_synonym_indicator':"同义",
           "is_pathology":"病理",
           "nurse":"护理",
           "report":"报告",
           "inspect":"观察",
           "non-medication_treat":"治疗",
           "surgical_treat":"治疗",
           "disease": "疾病",
           "drug": "药物",
           "surgery": "手术",
           "symptom": "症状",
           "prevention": "预防措施",
           "inspection_method": "观察方法",
           "diagnostic_technology": "诊断技术",
           "non-drug_therapy": "非药疗法",
           "nursing_technology": "护理技术",
           "cause_of_disease": "病因",
           "pathological_mechanism": "病理",
           'medical_indicator': '医疗指标',
           'has_subclass':'分为',
           'has_indicator':'检测项目为',
           'has_inspector':'检测项目为',
           'drug_effect':'作用于',
           'drug_effect_symptom':'作用于',
           'drug_effect_pathological':'作用于',
           'non-drug_therapy_effect':'作用于',
           'prevention_effect':'作用于',
           'drug_cause_of_disease':'作用于',
           'lead':"导致",
           'diagnostic_pathological':'诊断',
           'indicate_symptom':"揭示",
           'nursing_symptom':'护理',
           
          }
with codecs.open(r".\关系抽取\abstract_000.ann", "r", encoding="utf-8") as f:#读取标签文件
    line = f.readline()#获取第一行
    print(line)
    line = line.strip("\n\r")
    entity=[]#标注文件中的实体储存在此处
    relation=[]#标注文件中的关系储存在此处
    while line != "":
        line_arr = line.split()
        cls = tag_dic[line_arr[1]] #转换为中文标签
        line_arr[1]=cls#中文标签替换
        if "T" in line_arr[0]:
            entity.append(line_arr)#实体存入entity
        else:
            relation.append(line_arr)#关系存入relation
        print(line_arr)
        line = f.readline()#读取下一个
        line = line.strip("\n\r")
    relations=[]
    for i in range(len(relation)):
        for j in range(len(entity)):
            if entity[j][0]==relation[i][2][5:]:
                relation[i][2]=entity[j][2:]
            if entity[j][0]==relation[i][3][5:]:
                relation[i][3]=entity[j][2:]
                
                

T1	disease 250 257	阿尔茨海默氏病

['T1', '疾病', '250', '257', '阿尔茨海默氏病']
['T2', '症状', '283', '288', '进行性痴呆']
['T3', '病因', '354', '356', '遗传']
['R1', '临床表现', 'Arg1:T1', 'Arg2:T2']
['T4', '疾病', '24', '30', '阿尔茨海默病']
['T6', '药物', '713', '718', '补肾健脾方']
['T7', '疾病', '561', '567', '阿尔茨海默病']
['T8', '药物', '1520', '1523', '奎硫平']
['T9', '药物', '1524', '1527', '利培酮']
['T10', '疾病', '1528', '1534', '阿尔茨海默病']
['R5', '药物治疗', 'Arg1:T8', 'Arg2:T10']
['R6', '药物治疗', 'Arg1:T9', 'Arg2:T10']
['T11', '疾病', '2565', '2571', '阿尔茨海默病']
['T12', '病理', '2589', '2597', '大脑皮层厚度萎缩']
['T13', '症状', '2598', '2606', '认知功能逐步下降']
['R8', '临床表现', 'Arg1:T11', 'Arg2:T13']
['T14', '疾病', '4734', '4740', '阿尔兹海默症']
['T15', '症状', '4744', '4750', '智力缓慢丧失']
['R9', '临床表现', 'Arg1:T14', 'Arg2:T15']
['T16', '药物', '5317', '5320', '都可喜']
['T17', '药物', '5322', '5327', '补肾益智汤']
['T18', '疾病', '5329', '5334', '老年痴呆病']
['R11', '药物治疗', 'Arg1:T16', 'Arg2:T18']
['T19', '疾病', '2821', '2826', '老年痴呆症']
['T20', '预防措施', '2792', '2794', '绿茶']
['R12', '预防', 'Arg

['T504', '观察方法', '11186', '11194', '临床疗效总评量表']
['T1292', '观察方法', '11195', '11198', 'CGI']
['T511', '观察方法', '11208', '11218', '锥体外系不良反应量表']
['T1293', '观察方法', '11219', '11224', 'RSESE']
['T512', '观察方法', '11226', '11232', '不良反应量表']
['T1294', '观察方法', '11233', '11237', 'TESS']
['T801', '医疗指标', '11717', '11721', '精神状态']
['T522', '观察方法', '12120', '12132', '简易智能精神状态检查量表']
['T1295', '观察方法', '12133', '12137', 'MMSE']
['T523', '观察方法', '12139', '12146', '功能活动调查表']
['T1296', '观察方法', '12147', '12150', 'FAQ']
['T533', '医疗指标', '12108', '12112', '认知功能']
['T1297', '观察方法', '12152', '12160', '瑞文标准推理测验']
['T1298', '观察方法', '12161', '12164', 'SPM']
['T1299', '医疗指标', '12171', '12175', '认知功能']
['T1300', '医疗指标', '12191', '12195', '认知功能']
['T1301', '护理技术', '12234', '12236', '陪伴']
['T1302', '医疗指标', '12298', '12302', '认知功能']
['T1303', '医疗指标', '12227', '12231', '认知功能']
['T821', '病因', '12743', '12749', 'β淀粉样蛋白']
['T1304', '病因', '12750', '12758', 'amyloidβ']
['T1305', '病因', '12759', '12761', 'Aβ']
['T827', '病因', '128

In [55]:
relation

[['R1', '临床表现', ['250', '257', '阿尔茨海默氏病'], ['283', '288', '进行性痴呆']],
 ['R5', '药物治疗', ['1520', '1523', '奎硫平'], ['1528', '1534', '阿尔茨海默病']],
 ['R6', '药物治疗', ['1524', '1527', '利培酮'], ['1528', '1534', '阿尔茨海默病']],
 ['R8', '临床表现', ['2565', '2571', '阿尔茨海默病'], ['2598', '2606', '认知功能逐步下降']],
 ['R9', '临床表现', ['4734', '4740', '阿尔兹海默症'], ['4744', '4750', '智力缓慢丧失']],
 ['R11', '药物治疗', ['5317', '5320', '都可喜'], ['5329', '5334', '老年痴呆病']],
 ['R12', '预防', ['2792', '2794', '绿茶'], ['2821', '2826', '老年痴呆症']],
 ['R13', '病因', ['3070', '3075', '血管性因素'], ['3076', '3082', '阿尔茨海默病']],
 ['R14', '临床表现', ['3139', '3145', '阿尔茨海默病'], ['3169', '3174', '进行性痴呆']],
 ['R15', '药物治疗', ['13163', '13166', '美金刚'], ['13170', '13176', '阿尔茨海默病']],
 ['R18',
  '临床表现',
  ['13771', '13777', '阿尔茨海默病'],
  ['13829', '13840', '认知和记忆功能不断恶化']],
 ['R19',
  '临床表现',
  ['13771', '13777', '阿尔茨海默病'],
  ['13841', '13852', '日常生活能力进行性减退']],
 ['R20', '临床表现', ['13771', '13777', '阿尔茨海默病'], ['13857', '13863', '神经精神症状']],
 ['R21', '临床表现', ['13771', '137

In [56]:
with codecs.open(r".\关系抽取\abstract_000.txt", "r", encoding="utf-8") as f:#读取文本文档
        content_str = f.read()

            

In [57]:
content_str

'《PET/CT在胸水诊断中的应用》(关志伟);《阿尔茨海默病11C-PIB PET连续动态显像结果分析》(郭喆);《18F-FDG PET(PET/CT)肾癌显像特点分析》(徐白萱);《18F-FDG和18F-FLT PET/CT显像在肺结节治疗决策中的作\r\n老年痴呆,主要分为血管性痴呆(Vascular Disease,VD)和老年性痴呆(Alzheimer Disease,AD)两大类,其中AD病约占30%,VD约占60%,其余为混合性痴呆[1]。为了寻找治疗老年痴呆的药物,世界各国学者\r\n阿尔茨海默氏病(Alzheimer\'s Disease,AD)是以进行性痴呆为主要临床特征的大脑退行性疾病。l907年病理学家Alzheimer首先描述了AD病的组织学改变[1]。AD发病机制尚不清楚,可能与遗传、环\r\n阿尔茨海默病(Alzheimer\'s disease,AD)是一种神经系统退行性疾病,主要的病理特征是在大脑控制情绪和记忆的海马区,以及控制语言的皮质部位出现大量的老年斑和神经纤维团(NFTs)。目前AD的发病机制及药物研究已引起人们的高度重视。随着对AD致病机制的深入研究,治疗AD的药物开发研究不断取得进展。该文针对目前中草药及相关制剂治疗AD的研究进展作了简要的概述。\r\n目的研究中药补肾健脾方对阿尔茨海默病(AD)大鼠脑组织标本中细胞周期蛋白依赖性蛋白激酶5(cdk-5)复合物(cdk-5/p35)表达的影响。方法雄性SD大鼠随机分为5组:正常对照组,AD模型组,假手术组,AD大鼠补肾健脾方小剂量干预组及大剂量干预组,每组5只。采用鹅膏蕈氨酸毁损大鼠脑Meynert基底核制备AD动物模型,中药补肾健脾方分别以3.6g·kg-1.d-1、7.2g·kg-1.d-1灌胃。30d后以水迷宫测试大鼠学习及记忆能力,以RT-PCR、Western blot方法分别检测脑组织中cdk-5/p35基因和蛋白的表达。结果 AD大鼠模型脑组织中cdk-5/p35基因和蛋白的表达显著高于正常对照组;大剂量及小剂量干预组大鼠脑组织中cdk-5/p35基因和蛋白的表达较AD模型组显著下降。结论补肾健脾方可显著改善大鼠学习记忆功能,同时下调cdk-5/p35基因和蛋白的表达,提示cdk-5/p35可能参与了补肾健脾方治疗AD的作用机制。\r\n目的:观察独活乙

In [58]:
con=content_str.replace("\r\n","。。")

In [59]:
#将/r/n换成。方便处理同时不改变长度
import re
p1 = r"。+"
pattern1 = re.compile(p1)
print(pattern1.findall(con))
newKey = re.sub(p1, "。", con)
newKey

['。。', '。', '。。', '。', '。', '。。', '。', '。', '。', '。。。', '。', '。', '。', '。', '。', '。。。', '。', '。', '。', '。', '。', '。', '。', '。。。', '。。。', '。', '。', '。', '。', '。', '。。。', '。。', '。', '。', '。', '。。。', '。', '。。。', '。', '。', '。', '。。。', '。。。', '。', '。', '。。。。。', '。', '。', '。', '。。。', '。', '。。。', '。', '。', '。', '。', '。。。', '。。。', '。', '。', '。', '。。。', '。', '。', '。', '。', '。', '。', '。', '。', '。', '。', '。。。', '。', '。', '。', '。', '。。。', '。。', '。', '。', '。', '。。。', '。', '。', '。', '。', '。。。', '。', '。', '。', '。。。', '。', '。', '。', '。。。', '。', '。', '。', '。。。', '。', '。', '。', '。。。', '。', '。', '。', '。', '。。。', '。', '。', '。', '。', '。。。。。。。', '。', '。。。', '。', '。', '。。', '。', '。', '。', '。', '。', '。', '。。。', '。', '。', '。', '。。。', '。', '。', '。', '。', '。', '。。。', '。', '。', '。', '。。。。。', '。', '。', '。', '。', '。', '。', '。', '。。。。。', '。', '。', '。', '。。。', '。', '。', '。', '。', '。。。。。', '。', '。', '。', '。。。', '。', '。', '。', '。', '。。。', '。', '。', '。', '。。。', '。', '。', '。', '。。。', '。', '。', '。', '。。。', '。', '。', '。', 

'《PET/CT在胸水诊断中的应用》(关志伟);《阿尔茨海默病11C-PIB PET连续动态显像结果分析》(郭喆);《18F-FDG PET(PET/CT)肾癌显像特点分析》(徐白萱);《18F-FDG和18F-FLT PET/CT显像在肺结节治疗决策中的作。老年痴呆,主要分为血管性痴呆(Vascular Disease,VD)和老年性痴呆(Alzheimer Disease,AD)两大类,其中AD病约占30%,VD约占60%,其余为混合性痴呆[1]。为了寻找治疗老年痴呆的药物,世界各国学者。阿尔茨海默氏病(Alzheimer\'s Disease,AD)是以进行性痴呆为主要临床特征的大脑退行性疾病。l907年病理学家Alzheimer首先描述了AD病的组织学改变[1]。AD发病机制尚不清楚,可能与遗传、环。阿尔茨海默病(Alzheimer\'s disease,AD)是一种神经系统退行性疾病,主要的病理特征是在大脑控制情绪和记忆的海马区,以及控制语言的皮质部位出现大量的老年斑和神经纤维团(NFTs)。目前AD的发病机制及药物研究已引起人们的高度重视。随着对AD致病机制的深入研究,治疗AD的药物开发研究不断取得进展。该文针对目前中草药及相关制剂治疗AD的研究进展作了简要的概述。目的研究中药补肾健脾方对阿尔茨海默病(AD)大鼠脑组织标本中细胞周期蛋白依赖性蛋白激酶5(cdk-5)复合物(cdk-5/p35)表达的影响。方法雄性SD大鼠随机分为5组:正常对照组,AD模型组,假手术组,AD大鼠补肾健脾方小剂量干预组及大剂量干预组,每组5只。采用鹅膏蕈氨酸毁损大鼠脑Meynert基底核制备AD动物模型,中药补肾健脾方分别以3.6g·kg-1.d-1、7.2g·kg-1.d-1灌胃。30d后以水迷宫测试大鼠学习及记忆能力,以RT-PCR、Western blot方法分别检测脑组织中cdk-5/p35基因和蛋白的表达。结果 AD大鼠模型脑组织中cdk-5/p35基因和蛋白的表达显著高于正常对照组;大剂量及小剂量干预组大鼠脑组织中cdk-5/p35基因和蛋白的表达较AD模型组显著下降。结论补肾健脾方可显著改善大鼠学习记忆功能,同时下调cdk-5/p35基因和蛋白的表达,提示cdk-5/p35可能参与了补肾健脾方治疗AD的作用机制。目的:观察独活乙醇提取物对阿尔茨海默病(AD)模型

In [60]:
sen_full=newKey.split("。")

In [61]:
sen_full

['《PET/CT在胸水诊断中的应用》(关志伟);《阿尔茨海默病11C-PIB PET连续动态显像结果分析》(郭喆);《18F-FDG PET(PET/CT)肾癌显像特点分析》(徐白萱);《18F-FDG和18F-FLT PET/CT显像在肺结节治疗决策中的作',
 '老年痴呆,主要分为血管性痴呆(Vascular Disease,VD)和老年性痴呆(Alzheimer Disease,AD)两大类,其中AD病约占30%,VD约占60%,其余为混合性痴呆[1]',
 '为了寻找治疗老年痴呆的药物,世界各国学者',
 "阿尔茨海默氏病(Alzheimer's Disease,AD)是以进行性痴呆为主要临床特征的大脑退行性疾病",
 'l907年病理学家Alzheimer首先描述了AD病的组织学改变[1]',
 'AD发病机制尚不清楚,可能与遗传、环',
 "阿尔茨海默病(Alzheimer's disease,AD)是一种神经系统退行性疾病,主要的病理特征是在大脑控制情绪和记忆的海马区,以及控制语言的皮质部位出现大量的老年斑和神经纤维团(NFTs)",
 '目前AD的发病机制及药物研究已引起人们的高度重视',
 '随着对AD致病机制的深入研究,治疗AD的药物开发研究不断取得进展',
 '该文针对目前中草药及相关制剂治疗AD的研究进展作了简要的概述',
 '目的研究中药补肾健脾方对阿尔茨海默病(AD)大鼠脑组织标本中细胞周期蛋白依赖性蛋白激酶5(cdk-5)复合物(cdk-5/p35)表达的影响',
 '方法雄性SD大鼠随机分为5组:正常对照组,AD模型组,假手术组,AD大鼠补肾健脾方小剂量干预组及大剂量干预组,每组5只',
 '采用鹅膏蕈氨酸毁损大鼠脑Meynert基底核制备AD动物模型,中药补肾健脾方分别以3.6g·kg-1.d-1、7.2g·kg-1.d-1灌胃',
 '30d后以水迷宫测试大鼠学习及记忆能力,以RT-PCR、Western blot方法分别检测脑组织中cdk-5/p35基因和蛋白的表达',
 '结果 AD大鼠模型脑组织中cdk-5/p35基因和蛋白的表达显著高于正常对照组;大剂量及小剂量干预组大鼠脑组织中cdk-5/p35基因和蛋白的表达较AD模型组显著下降',
 '结论补肾健脾方可显著改善大鼠学习记忆功能,同时下调cdk-5/p35基因和蛋白的

In [62]:
#模型在处理是会有一个中间状态是set所以西文括号会报错
cleaned_sen=[]
for i in sen_full:
    i=i.replace("(",'')
    i=i.replace(")",'')
    i=i.replace(" ",'')
    i=i.replace("Alzheimer's","Alzheimer'sdisease")
    cleaned_sen.append(i)

In [63]:
cleaned_sen

['《PET/CT在胸水诊断中的应用》关志伟;《阿尔茨海默病11C-PIBPET连续动态显像结果分析》郭喆;《18F-FDGPETPET/CT肾癌显像特点分析》徐白萱;《18F-FDG和18F-FLTPET/CT显像在肺结节治疗决策中的作',
 '老年痴呆,主要分为血管性痴呆VascularDisease,VD和老年性痴呆AlzheimerDisease,AD两大类,其中AD病约占30%,VD约占60%,其余为混合性痴呆[1]',
 '为了寻找治疗老年痴呆的药物,世界各国学者',
 "阿尔茨海默氏病Alzheimer'sdiseaseDisease,AD是以进行性痴呆为主要临床特征的大脑退行性疾病",
 'l907年病理学家Alzheimer首先描述了AD病的组织学改变[1]',
 'AD发病机制尚不清楚,可能与遗传、环',
 "阿尔茨海默病Alzheimer'sdiseasedisease,AD是一种神经系统退行性疾病,主要的病理特征是在大脑控制情绪和记忆的海马区,以及控制语言的皮质部位出现大量的老年斑和神经纤维团NFTs",
 '目前AD的发病机制及药物研究已引起人们的高度重视',
 '随着对AD致病机制的深入研究,治疗AD的药物开发研究不断取得进展',
 '该文针对目前中草药及相关制剂治疗AD的研究进展作了简要的概述',
 '目的研究中药补肾健脾方对阿尔茨海默病AD大鼠脑组织标本中细胞周期蛋白依赖性蛋白激酶5cdk-5复合物cdk-5/p35表达的影响',
 '方法雄性SD大鼠随机分为5组:正常对照组,AD模型组,假手术组,AD大鼠补肾健脾方小剂量干预组及大剂量干预组,每组5只',
 '采用鹅膏蕈氨酸毁损大鼠脑Meynert基底核制备AD动物模型,中药补肾健脾方分别以3.6g·kg-1.d-1、7.2g·kg-1.d-1灌胃',
 '30d后以水迷宫测试大鼠学习及记忆能力,以RT-PCR、Westernblot方法分别检测脑组织中cdk-5/p35基因和蛋白的表达',
 '结果AD大鼠模型脑组织中cdk-5/p35基因和蛋白的表达显著高于正常对照组;大剂量及小剂量干预组大鼠脑组织中cdk-5/p35基因和蛋白的表达较AD模型组显著下降',
 '结论补肾健脾方可显著改善大鼠学习记忆功能,同时下调cdk-5/p35基因和蛋白的表达,提示cdk-5/p35可能参与了

In [64]:
#找出每个句号索引
full_mark_list=[]
for i,full_mark in enumerate(con):
    if (full_mark=="。") :
        full_mark_list.append(i)

In [65]:
#排除掉连续句号，之前替换导致的
del_li=[]
for i in range(len(full_mark_list)):
    try:
        if full_mark_list[i+1]-full_mark_list[i]==1:
            del_li.append(full_mark_list[i+1])
    except:
        pass

In [66]:
len(del_li)

190

In [67]:
#所以句号位置 方便分句
cleaned_li=[]
for i in full_mark_list:
    if i not in del_li:
        cleaned_li.append(i)

In [68]:
cleaned_li

[128,
 227,
 248,
 303,
 339,
 358,
 457,
 482,
 515,
 546,
 619,
 679,
 750,
 818,
 902,
 976,
 1059,
 1086,
 1132,
 1192,
 1217,
 1276,
 1386,
 1440,
 1512,
 1559,
 1603,
 1669,
 1698,
 1713,
 1747,
 2003,
 2051,
 2182,
 2280,
 2332,
 2419,
 2562,
 2606,
 2674,
 2712,
 2755,
 2844,
 2889,
 2942,
 2975,
 3015,
 3053,
 3095,
 3136,
 3201,
 3252,
 3283,
 3316,
 3391,
 3456,
 3494,
 3589,
 3614,
 3717,
 3756,
 3821,
 3891,
 3931,
 3956,
 3983,
 4025,
 4064,
 4087,
 4123,
 4150,
 4223,
 4255,
 4282,
 4362,
 4501,
 4677,
 4731,
 4976,
 5059,
 5122,
 5249,
 5307,
 5339,
 5366,
 5397,
 5427,
 5458,
 5522,
 5582,
 5665,
 5713,
 5777,
 5851,
 5941,
 6011,
 6082,
 6179,
 6332,
 6363,
 6402,
 6445,
 6664,
 6721,
 6833,
 6881,
 6905,
 6997,
 7066,
 7122,
 7158,
 7263,
 7369,
 7443,
 7558,
 7566,
 7596,
 7699,
 7738,
 7770,
 7804,
 7846,
 7861,
 7923,
 8004,
 8052,
 8087,
 8169,
 8238,
 8306,
 8331,
 8396,
 8431,
 8463,
 8486,
 8513,
 8587,
 8632,
 8667,
 8727,
 8821,
 8863,
 9039,
 9080,
 9142,
 

In [69]:
#index找出关系及对应的句子
relation_sen=[]
for i in relation:
    i[2][2]=i[2][2].replace("(",'')
    i[2][2]=i[2][2].replace("(",'')
    i[2][2]=i[2][2].replace(" ",'')
    i[2][2]=i[2][2].replace("Alzheimer's","Alzheimer'sdisease")
    i[3][2]=i[3][2].replace(")",'')
    i[3][2]=i[3][2].replace(")",'')
    i[3][2]=i[3][2].replace(" ",'')
    i[3][2]=i[3][2].replace("Alzheimer's","Alzheimer'sdisease")
    try:
        for j in range(len(cleaned_li)):
                if int(i[2][0])>int(cleaned_li[j]) and int(i[2][0])<int(cleaned_li[j+1]) and int(i[3][0])>int(cleaned_li[j]) and int(i[3][0])<int(cleaned_li[j+1]):
                    relation_sen.append([i[1],i[2][2],i[3][2],j,cleaned_sen[j+1]])#中间的句子也要放进去 我需要句子的index
    except:
        pass

In [70]:
relation_sen

[['临床表现',
  '阿尔茨海默氏病',
  '进行性痴呆',
  2,
  "阿尔茨海默氏病Alzheimer'sdiseaseDisease,AD是以进行性痴呆为主要临床特征的大脑退行性疾病"],
 ['药物治疗', '奎硫平', '阿尔茨海默病', 24, '目的:比较奎硫平与利培酮对阿尔茨海默病AD患者精神行为症状BPSD的疗效和安全性'],
 ['药物治疗', '利培酮', '阿尔茨海默病', 24, '目的:比较奎硫平与利培酮对阿尔茨海默病AD患者精神行为症状BPSD的疗效和安全性'],
 ['临床表现',
  '阿尔茨海默病',
  '认知功能逐步下降',
  37,
  '阿尔茨海默病是一种进行性神经退化痴呆症,其特征是大脑皮层厚度萎缩和认知功能逐步下降'],
 ['临床表现',
  '阿尔兹海默症',
  '智力缓慢丧失',
  77,
  '阿尔兹海默症是一种以智力缓慢丧失为特征的神经退行性疾病,主要病理特征之一为神经细胞内出现大量神经纤维缠结.神经纤维缠结的主要成分为异常磷酸化的Tau蛋白,研究Tau蛋白的结构和功能以及神经纤维缠结形成的分子机制具有重要的生理意义.Tau和Aβ是阿尔兹海默病药物开发的两个重要靶点,随着近期一些以Aβ为靶点的药物临床研究宣告失败,越来越多的研究组已经开始考虑改变研发阿尔兹海默病药物的策略,并开始转向以Tau为靶点的药物开发.本文综述了Tau介导阿尔兹海默病的机理及相关药物的研究进展.'],
 ['药物治疗', '都可喜', '老年痴呆病', 82, '目的:观察西药都可喜联合补肾益智汤治疗老年痴呆病的临床疗效'],
 ['预防',
  '绿茶',
  '老年痴呆症',
  41,
  '德国马克斯·德尔布吕克分子医学中心14日报告的一项最新研究成果显示,绿茶中活性物质EGCG表没食子儿茶素没食子酸酯可解除与老年痴呆症等疾病有关的蛋白质异常沉积带来的毒性'],
 ['病因', '血管性因素', '阿尔茨海默病', 47, '结果高龄为老年期痴呆的决定因素,血管性因素是阿尔茨海默病、血管性痴呆共同的危险因素'],
 ['临床表现',
  '阿尔茨海默病',
  '进行性痴呆',
  49,
  '阿尔茨海默病Alzheimerdisease,AD是以进行性痴呆为特征的大脑退行性变性疾病,目前治疗缺乏特异有效的手段

In [71]:
#检查是否关系是否在句子中
for i in relation_sen:
    if i[1] in i[4] and i[2] in i[4]:
        print('ture')
    else:
        print("false",i)

ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
false ['诊断', '表面加强激光解析电离飞行时间质谱SELDI-TOF-MS)蛋白质芯片', '阿尔茨海默病', 78, '目的用表面加强激光解析电离飞行时间质谱SELDI-TOF-MS蛋白质芯片检测阿尔茨海默病AD患者血清蛋白质指纹图谱,探讨诊断模型在AD诊断中的临床应用价值']
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture
ture

In [72]:
#保存
w_path="./train.txt"
with codecs.open(w_path, "w", encoding="utf-8") as w:
    for i in relation_sen:
        w.write(i[1]+"\t"+i[2]+"\t"+i[0]+"\t"+i[4])
        w.write("\r\n")

In [73]:
#将所有括号替换

In [109]:
with codecs.open(w_path, "r", encoding="utf-8") as f:
    for line in f:
        chunks=line.split()
        if len(tuple(chunks)) !=4:
            print(tuple(chunks))

In [110]:
a='《PET/CT在胸水诊断中的应用》(关志伟);《阿尔茨海默病11C-PIB PET连续动态显像结果分析》(郭喆);《18F-FDG PET(PET/CT)肾癌显像特点分析》(徐白萱);《18F-FDG和18F-FLT PET/CT显像在肺结节治疗决策中的作'

In [111]:
a.replace("(",'')

'《PET/CT在胸水诊断中的应用》关志伟);《阿尔茨海默病11C-PIB PET连续动态显像结果分析》郭喆);《18F-FDG PETPET/CT)肾癌显像特点分析》徐白萱);《18F-FDG和18F-FLT PET/CT显像在肺结节治疗决策中的作'

In [112]:
rel_li=[]
for i in relation:
    if i[1] not in rel_li:
        rel_li.append(i[1])

In [113]:
rel_li

['临床表现',
 '药物治疗',
 '预防',
 '病因',
 '同义',
 '病理',
 '护理',
 '诊断',
 '报告',
 '非药治疗',
 '观察',
 '检测项目为',
 '作用于',
 '导致',
 '揭示',
 '分为',
 '并发症']

制作负样本

In [114]:
entity

[['T1', '疾病', '250', '257', '阿尔茨海默氏病'],
 ['T2', '症状', '283', '288', '进行性痴呆'],
 ['T3', '病因', '354', '356', '遗传'],
 ['T4', '疾病', '24', '30', '阿尔茨海默病'],
 ['T6', '药物', '713', '718', '补肾健脾方'],
 ['T7', '疾病', '561', '567', '阿尔茨海默病'],
 ['T8', '药物', '1520', '1523', '奎硫平'],
 ['T9', '药物', '1524', '1527', '利培酮'],
 ['T10', '疾病', '1528', '1534', '阿尔茨海默病'],
 ['T11', '疾病', '2565', '2571', '阿尔茨海默病'],
 ['T12', '病理', '2589', '2597', '大脑皮层厚度萎缩'],
 ['T13', '症状', '2598', '2606', '认知功能逐步下降'],
 ['T14', '疾病', '4734', '4740', '阿尔兹海默症'],
 ['T15', '症状', '4744', '4750', '智力缓慢丧失'],
 ['T16', '药物', '5317', '5320', '都可喜'],
 ['T17', '药物', '5322', '5327', '补肾益智汤'],
 ['T18', '疾病', '5329', '5334', '老年痴呆病'],
 ['T19', '疾病', '2821', '2826', '老年痴呆症'],
 ['T20', '预防措施', '2792', '2794', '绿茶'],
 ['T21', '疾病', '3076', '3082', '阿尔茨海默病'],
 ['T22', '病因', '3070', '3075', '血管性因素'],
 ['T23', '疾病', '3139', '3145', '阿尔茨海默病'],
 ['T24', '症状', '3169', '3174', '进行性痴呆'],
 ['T25', '药物', '13163', '13166', '美金刚'],
 ['T26', '疾病', '13170', '13176',

In [115]:
#制作负样本 找出句子中所有的实体
sen_for_unkn=[]
for i in range(len(cleaned_sen)):
    sen_for_unkn.append([cleaned_sen[i]])
for i in entity:
    for j in range(len(cleaned_li)):
        if int(i[2])>int(cleaned_li[j]) and int(i[2])<int(cleaned_li[j+1]):
            sen_for_unkn[j+1].append(i[4])#中间的句子也要放进去 我需要句子的index

In [116]:
sen_for_unkn

[['《PET/CT在胸水诊断中的应用》关志伟;《阿尔茨海默病11C-PIBPET连续动态显像结果分析》郭喆;《18F-FDGPETPET/CT肾癌显像特点分析》徐白萱;《18F-FDG和18F-FLTPET/CT显像在肺结节治疗决策中的作'],
 ['老年痴呆,主要分为血管性痴呆VascularDisease,VD和老年性痴呆AlzheimerDisease,AD两大类,其中AD病约占30%,VD约占60%,其余为混合性痴呆[1]',
  '老年痴呆',
  '血管性痴呆',
  '老年性痴呆',
  'Alzheimer',
  'AD',
  '混合性痴呆',
  'AD',
  'Vascular',
  'VD',
  'VD'],
 ['为了寻找治疗老年痴呆的药物,世界各国学者', '老年痴呆'],
 ["阿尔茨海默氏病Alzheimer'sdiseaseDisease,AD是以进行性痴呆为主要临床特征的大脑退行性疾病",
  '阿尔茨海默氏病',
  '进行性痴呆',
  "Alzheimer's",
  'AD'],
 ['l907年病理学家Alzheimer首先描述了AD病的组织学改变[1]', 'AD'],
 ['AD发病机制尚不清楚,可能与遗传、环', '遗传', 'AD'],
 ["阿尔茨海默病Alzheimer'sdiseasedisease,AD是一种神经系统退行性疾病,主要的病理特征是在大脑控制情绪和记忆的海马区,以及控制语言的皮质部位出现大量的老年斑和神经纤维团NFTs",
  '阿尔茨海默病',
  "Alzheimer's",
  'AD',
  '老年斑',
  '神经纤维团',
  'NFTs'],
 ['目前AD的发病机制及药物研究已引起人们的高度重视', 'AD'],
 ['随着对AD致病机制的深入研究,治疗AD的药物开发研究不断取得进展', 'AD', 'AD'],
 ['该文针对目前中草药及相关制剂治疗AD的研究进展作了简要的概述', 'AD'],
 ['目的研究中药补肾健脾方对阿尔茨海默病AD大鼠脑组织标本中细胞周期蛋白依赖性蛋白激酶5cdk-5复合物cdk-5/p35表达的影响',
  '阿尔茨海默病',
  '补肾健脾方',
  'AD',
  '细胞周期蛋白依赖性蛋白激酶5',
  'cdk-5',
  'cd

In [117]:
#找出已存在的关系对
relation_pair=[]
for i in relation_sen:
    if [i[1],i[2]] not in relation_pair:
        relation_pair.append([i[1],i[2]])

In [118]:
len(relation_pair)

660

In [119]:
#解决关系中出现同义但未画联系的问题
syn_word=[]
for i in relation:
    if i[1]=="同义":
        syn_word.append([i[2][2],i[3][2]])

In [120]:
syn_word

[['阿尔茨海默病', "Alzheimer'sdisease"],
 ["Alzheimer'sdisease", 'AD'],
 ['阿尔茨海默病', 'AD'],
 ["Alzheimer'sdisease", '阿尔茨海默病'],
 ['AD', '阿尔茨海默病'],
 ['AD', "Alzheimer'sdisease"],
 ['阿尔茨海默病', 'AD'],
 ['Alzheimer', '老年性痴呆'],
 ['AD', '老年性痴呆'],
 ['AD', '阿尔茨海默病'],
 ['AD', '阿尔茨海默病'],
 ["Alzheimer'sdisease", '阿尔茨海默氏病'],
 ['AD', '阿尔茨海默氏病'],
 ['AD', '阿尔茨海默病'],
 ['AD', '阿尔茨海默病'],
 ['AD', '阿尔茨海默症'],
 ['AD', '阿尔茨海默病'],
 ['AD', '阿尔茨海默病'],
 ['AD', '老年性痴呆'],
 ['AD', 'Alzheimer病'],
 ['AD', '阿尔茨海默病'],
 ['AD', '阿尔茨海默病'],
 ['阿尔茨海默病', 'AD'],
 ['SAD', '散发性阿尔茨海默病'],
 ['Alzheimer', '阿尔茨海默病'],
 ['AD', '阿尔茨海默病'],
 ['AD', '阿尔茨海默病'],
 ['AD', '阿尔茨海默病'],
 ['Alzheimer’s', '阿尔茨海默病'],
 ['AD', '阿尔茨海默病'],
 ['血管性痴呆', 'VD'],
 ['Alzheimer', '老年性痴呆'],
 ['AD', '老年性痴呆'],
 ['Alzheimer’s', '阿尔茨海默病'],
 ['AD', '阿尔茨海默病'],
 ["Alzheimer'sdisease", '阿尔茨海默病'],
 ['AD', '阿尔茨海默病'],
 ['AD', '阿尔茨海默病'],
 ['AD', '阿尔茨海默病'],
 ['AD', '阿尔茨海默病'],
 ['Alzheimer’s', '阿尔茨海默病'],
 ['AD', '阿尔茨海默病'],
 ['AD', '阿尔茨海默病'],
 ['AD', '阿尔茨海默病'],
 ["Alzheimer'sdisease", 

In [121]:
#生成同义替换的关系对 之后从候选集中排除
relat_plus_syn=[]
for i in relation_pair:
    for j in syn_word:
        if i[0] ==j[0]:
            relat_plus_syn.append([j[1],i[1]])
        if i[1] ==j[0]:
            relat_plus_syn.append([i[0],j[1]])
        if i[0] ==j[1]:
            relat_plus_syn.append([j[0],i[1]])
        if i[1] ==j[1]:
            relat_plus_syn.append([i[0],j[0]])

In [122]:
relat_plus_syn

[["Alzheimer'sdisease", '进行性痴呆'],
 ['AD', '进行性痴呆'],
 ['奎硫平', "Alzheimer'sdisease"],
 ['奎硫平', 'AD'],
 ['奎硫平', "Alzheimer'sdisease"],
 ['奎硫平', 'AD'],
 ['奎硫平', 'AD'],
 ['奎硫平', 'AD'],
 ['奎硫平', 'AD'],
 ['奎硫平', 'AD'],
 ['奎硫平', 'AD'],
 ['奎硫平', 'AD'],
 ['奎硫平', 'AD'],
 ['奎硫平', 'AD'],
 ['奎硫平', 'AD'],
 ['奎硫平', 'AD'],
 ['奎硫平', 'Alzheimer'],
 ['奎硫平', 'AD'],
 ['奎硫平', 'AD'],
 ['奎硫平', 'AD'],
 ['奎硫平', 'Alzheimer’s'],
 ['奎硫平', 'AD'],
 ['奎硫平', 'Alzheimer’s'],
 ['奎硫平', 'AD'],
 ['奎硫平', "Alzheimer'sdisease"],
 ['奎硫平', 'AD'],
 ['奎硫平', 'AD'],
 ['奎硫平', 'AD'],
 ['奎硫平', 'AD'],
 ['奎硫平', 'Alzheimer’s'],
 ['奎硫平', 'AD'],
 ['奎硫平', 'AD'],
 ['奎硫平', 'AD'],
 ['奎硫平', "Alzheimer'sdisease"],
 ['奎硫平', 'AD'],
 ['奎硫平', 'AD'],
 ['奎硫平', 'AD'],
 ['奎硫平', 'AD'],
 ['奎硫平', 'AD'],
 ['奎硫平', 'AD'],
 ['奎硫平', 'AD'],
 ['奎硫平', 'AD'],
 ['利培酮', "Alzheimer'sdisease"],
 ['利培酮', 'AD'],
 ['利培酮', "Alzheimer'sdisease"],
 ['利培酮', 'AD'],
 ['利培酮', 'AD'],
 ['利培酮', 'AD'],
 ['利培酮', 'AD'],
 ['利培酮', 'AD'],
 ['利培酮', 'AD'],
 ['利培酮', 'AD'],
 ['利培酮', 'AD'],
 [

In [123]:
plused_reli=relation_pair+relat_plus_syn

In [124]:
plused_reli

[['阿尔茨海默氏病', '进行性痴呆'],
 ['奎硫平', '阿尔茨海默病'],
 ['利培酮', '阿尔茨海默病'],
 ['阿尔茨海默病', '认知功能逐步下降'],
 ['阿尔兹海默症', '智力缓慢丧失'],
 ['都可喜', '老年痴呆病'],
 ['绿茶', '老年痴呆症'],
 ['血管性因素', '阿尔茨海默病'],
 ['阿尔茨海默病', '进行性痴呆'],
 ['美金刚', '阿尔茨海默病'],
 ['阿尔茨海默病', '认知和记忆功能不断恶化'],
 ['阿尔茨海默病', '日常生活能力进行性减退'],
 ['阿尔茨海默病', '神经精神症状'],
 ['阿尔茨海默病', '行为障碍'],
 ['多奈哌齐', '老年痴呆'],
 ['花青素', '阿尔茨海默症'],
 ['阿尔茨海默病', "Alzheimer'sdisease"],
 ["Alzheimer'sdisease", 'AD'],
 ['阿尔茨海默病', 'AD'],
 ['老年斑', '阿尔茨海默病'],
 ['补肾健脾方', '阿尔茨海默病'],
 ['自我管理教育', '阿尔茨海默病'],
 ['11C-PIB', '阿尔茨海默病'],
 ["Alzheimer'sdisease", '阿尔茨海默病'],
 ['AD', '阿尔茨海默病'],
 ['AD', "Alzheimer'sdisease"],
 ['同型半胱氨酸', '阿尔茨海默病'],
 ['叶酸', '阿尔茨海默病'],
 ['维生素B12', '阿尔茨海默病'],
 ['Alzheimer', '老年性痴呆'],
 ['AD', '老年性痴呆'],
 ["Alzheimer'sdisease", '阿尔茨海默氏病'],
 ['AD', '阿尔茨海默氏病'],
 ['AD', '阿尔茨海默症'],
 ['神经纤维缠结', '阿尔兹海默症'],
 ['表面加强激光解析电离飞行时间质谱SELDI-TOF-MS)蛋白质芯片', '阿尔茨海默病'],
 ['血糖', '阿尔茨海默病'],
 ['真胰岛素', '阿尔茨海默病'],
 ['针刺', '阿尔茨海默病'],
 ['AD', 'Alzheimer病'],
 ['护理管理', '老年痴呆'],
 ['意外防范护理', '老年痴呆'],
 ['β淀粉样多肽沉积

In [130]:
unkn_li=[]
for i in sen_for_unkn:
    for j in i[1:]:
        for k in i[1:]:
            if j!=k and(([j,k]  not in plused_reli) and ([k,j] not in plused_reli)) and (j!="老年痴呆" or j!='AD') and (k!="老年痴呆" or k!='AD'):
                unkn_li.append([j,k,i[0]])

In [131]:
len(unkn_li)

3696

In [132]:
import random
sample=random.sample(unkn_li, 500)

In [133]:
sample

[['水分',
  '体重',
  '结果与对照组女性比较,AD女性患者的体重、瘦组织、水分、蛋白质、矿物质均较低P<0.05;与对照组男性比较,AD男性患者体内的蛋白质和矿物质较低P<0.05'],
 ['MDA',
  'Aβ40/Aβ42',
  '为探讨不对称二甲基精氨酸ADMA含量及氧化应激与AD发病的关系,抽取34例正常人及37例AD患者脑脊液样本,比较分析了脑脊液内ADMA、Aβ40/Aβ42蛋白比例、一氧化氮NO、诱导型一氧化氮合成酶iNOS活性及丙二醛MDA、3-硝基酪氨酸3-NT、过氧亚硝基ONOO-的含量差异,分析了ADMA含量与诸因素间及其与AD症状的相关性,以阐明其与AD发病的关系'],
 ['远志',
  '兴奋性突触后电位',
  '结果模型组大鼠的学习记忆能力、兴奋性突触后电位与正常组相比明显下降P<0.01;远志高、低剂量组大鼠的学习记忆能力、兴奋性突触后电位与模型组相比均明显提高P<0.01或P<0.05;远志低剂量组大鼠的学习记忆能力、兴奋性突触后电位较远志高剂量组明显降低P<0.01'],
 ['细胞损伤',
  'H2O2',
  '氧化胁迫在阿尔茨海默症AD的发病过程中起重要作用.花青素是一种广泛存在于植物中的黄酮类物质.实验结果表明,100μmol/L的花青素可以有效地缓解AD模型细胞氧化胁迫,具体表现为,降低AD模型细胞的H2O2易感性,减少胞内ROS和[NO]i.c-Jun氨基端激酶JNK是氧化胁迫导致细胞损伤的重要信号途径.实验结果显示花青素可以有效地抑制氧化胁迫对JNK的激活.提示花青素的抗氧化胁迫作用与JNK信号途径有关.因此,花青素可以作为一种氧化胁迫因子的清除剂来保护AD模型细胞,有望用于AD的辅助治疗.'],
 ['锥体外系不良反应量表', 'TESS', '采用锥体外系不良反应量表RSESE、不良反应量表TESS评定药物不良反应,并进行比较分析'],
 ['错误次数',
  '锥体细胞数',
  '结果与模型组比较,银杏叶组动物潜伏期明显延长,错误次数降低P<0.05;银杏叶组动物脑组织SOD活性明显升高P<0.05,MDA含量、Ca+浓度含量、乙酰胆碱酯酶活性显著性降低P<0.05,与对照及脑复康组比较未见显著差异P>0.05;银杏叶组动物大脑海马锥体细胞数与模型组比较明显升高P<0.01,大脑皮质神

In [134]:
w_path="./unkwn.txt"
with codecs.open(w_path, "w", encoding="utf-8") as w:
    for i in sample:
        w.write(i[0]+"\t"+i[1]+"\t"+"unknown"+"\t"+i[2])
        w.write("\r\n")

知识图谱

In [48]:
graph_li=[]
for i in relation_sen:
    for j in entity:
        for k in entity:
            if i[1]==j[4] and i[2]==k[4]:
                graph_li.append([j[1],i[1],i[2],k[1],i[0]])

In [91]:
graph_li

[['疾病', '阿尔茨海默氏病', '进行性痴呆', '症状', '临床表现'],
 ['疾病', '阿尔茨海默氏病', '进行性痴呆', '症状', '临床表现'],
 ['疾病', '阿尔茨海默氏病', '进行性痴呆', '症状', '临床表现'],
 ['疾病', '阿尔茨海默氏病', '进行性痴呆', '症状', '临床表现'],
 ['药物', '奎硫平', '阿尔茨海默病', '疾病', '药物治疗'],
 ['药物', '奎硫平', '阿尔茨海默病', '疾病', '药物治疗'],
 ['药物', '奎硫平', '阿尔茨海默病', '疾病', '药物治疗'],
 ['药物', '奎硫平', '阿尔茨海默病', '疾病', '药物治疗'],
 ['药物', '奎硫平', '阿尔茨海默病', '疾病', '药物治疗'],
 ['药物', '奎硫平', '阿尔茨海默病', '疾病', '药物治疗'],
 ['药物', '奎硫平', '阿尔茨海默病', '疾病', '药物治疗'],
 ['药物', '奎硫平', '阿尔茨海默病', '疾病', '药物治疗'],
 ['药物', '奎硫平', '阿尔茨海默病', '疾病', '药物治疗'],
 ['药物', '奎硫平', '阿尔茨海默病', '疾病', '药物治疗'],
 ['药物', '奎硫平', '阿尔茨海默病', '疾病', '药物治疗'],
 ['药物', '奎硫平', '阿尔茨海默病', '疾病', '药物治疗'],
 ['药物', '奎硫平', '阿尔茨海默病', '疾病', '药物治疗'],
 ['药物', '奎硫平', '阿尔茨海默病', '疾病', '药物治疗'],
 ['药物', '奎硫平', '阿尔茨海默病', '疾病', '药物治疗'],
 ['药物', '奎硫平', '阿尔茨海默病', '疾病', '药物治疗'],
 ['药物', '奎硫平', '阿尔茨海默病', '疾病', '药物治疗'],
 ['药物', '奎硫平', '阿尔茨海默病', '疾病', '药物治疗'],
 ['药物', '奎硫平', '阿尔茨海默病', '疾病', '药物治疗'],
 ['药物', '奎硫平', '阿尔茨海默病', '疾病', '药物治疗'],
 ['药物', '奎硫平', '阿尔茨海默病', '疾病

In [92]:
new_li=[]
for i in graph_li:
    new_li.append({"headtype":i[0],"head":i[1],"tailtype":i[3],"tail":i[2],"relation":i[4]})

In [95]:
new_li

[{'headtype': '疾病',
  'head': '阿尔茨海默氏病',
  'tailtype': '症状',
  'tail': '进行性痴呆',
  'relation': '临床表现'},
 {'headtype': '疾病',
  'head': '阿尔茨海默氏病',
  'tailtype': '症状',
  'tail': '进行性痴呆',
  'relation': '临床表现'},
 {'headtype': '疾病',
  'head': '阿尔茨海默氏病',
  'tailtype': '症状',
  'tail': '进行性痴呆',
  'relation': '临床表现'},
 {'headtype': '疾病',
  'head': '阿尔茨海默氏病',
  'tailtype': '症状',
  'tail': '进行性痴呆',
  'relation': '临床表现'},
 {'headtype': '药物',
  'head': '奎硫平',
  'tailtype': '疾病',
  'tail': '阿尔茨海默病',
  'relation': '药物治疗'},
 {'headtype': '药物',
  'head': '奎硫平',
  'tailtype': '疾病',
  'tail': '阿尔茨海默病',
  'relation': '药物治疗'},
 {'headtype': '药物',
  'head': '奎硫平',
  'tailtype': '疾病',
  'tail': '阿尔茨海默病',
  'relation': '药物治疗'},
 {'headtype': '药物',
  'head': '奎硫平',
  'tailtype': '疾病',
  'tail': '阿尔茨海默病',
  'relation': '药物治疗'},
 {'headtype': '药物',
  'head': '奎硫平',
  'tailtype': '疾病',
  'tail': '阿尔茨海默病',
  'relation': '药物治疗'},
 {'headtype': '药物',
  'head': '奎硫平',
  'tailtype': '疾病',
  'tail': '阿尔茨海默病',
  'relation

In [97]:
unq_li=[]
for i in new_li:
    if i not in unq_li:
        unq_li.append(i)

In [98]:
unq_li

[{'headtype': '疾病',
  'head': '阿尔茨海默氏病',
  'tailtype': '症状',
  'tail': '进行性痴呆',
  'relation': '临床表现'},
 {'headtype': '药物',
  'head': '奎硫平',
  'tailtype': '疾病',
  'tail': '阿尔茨海默病',
  'relation': '药物治疗'},
 {'headtype': '药物',
  'head': '利培酮',
  'tailtype': '疾病',
  'tail': '阿尔茨海默病',
  'relation': '药物治疗'},
 {'headtype': '疾病',
  'head': '阿尔茨海默病',
  'tailtype': '症状',
  'tail': '认知功能逐步下降',
  'relation': '临床表现'},
 {'headtype': '疾病',
  'head': '阿尔兹海默症',
  'tailtype': '症状',
  'tail': '智力缓慢丧失',
  'relation': '临床表现'},
 {'headtype': '药物',
  'head': '都可喜',
  'tailtype': '疾病',
  'tail': '老年痴呆病',
  'relation': '药物治疗'},
 {'headtype': '预防措施',
  'head': '绿茶',
  'tailtype': '疾病',
  'tail': '老年痴呆症',
  'relation': '预防'},
 {'headtype': '病因',
  'head': '血管性因素',
  'tailtype': '疾病',
  'tail': '阿尔茨海默病',
  'relation': '病因'},
 {'headtype': '疾病',
  'head': '阿尔茨海默病',
  'tailtype': '症状',
  'tail': '进行性痴呆',
  'relation': '临床表现'},
 {'headtype': '药物',
  'head': '美金刚',
  'tailtype': '疾病',
  'tail': '阿尔茨海默病',
  'relation':

In [101]:
import pandas as pd

In [103]:
df=pd.DataFrame(unq_li)

In [105]:
df

,head,headtype,relation,tail,tailtype
0,阿尔茨海默氏病,疾病,临床表现,进行性痴呆,症状
1,奎硫平,药物,药物治疗,阿尔茨海默病,疾病
2,利培酮,药物,药物治疗,阿尔茨海默病,疾病
3,阿尔茨海默病,疾病,临床表现,认知功能逐步下降,症状
4,阿尔兹海默症,疾病,临床表现,智力缓慢丧失,症状
5,都可喜,药物,药物治疗,老年痴呆病,疾病
6,绿茶,预防措施,预防,老年痴呆症,疾病
7,血管性因素,病因,病因,阿尔茨海默病,疾病
8,阿尔茨海默病,疾病,临床表现,进行性痴呆,症状
9,美金刚,药物,药物治疗,阿尔茨海默病,疾病


In [106]:
df.to_csv("relation.csv")